<a href="https://colab.research.google.com/github/buraktalhaakin/MedSimulator/blob/main/new_model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ================================================================
# 1. KURULUM (VERSİYONLARI ÇİVİLİYORUZ - KESİN ÇÖZÜM)
# ================================================================
# Bu versiyonlar birbirleriyle %100 uyumludur. Hata verme şansı yok.
!pip install -q -U "huggingface_hub>=0.23.0"
!pip install -q -U "transformers==4.41.2"
!pip install -q -U "trl==0.8.6"
!pip install -q -U "peft==0.11.1"
!pip install -q -U "accelerate==0.30.1"
!pip install -q -U bitsandbytes

import torch
import json
import os
from huggingface_hub import login
from datasets import Dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    TrainerCallback
)
from trl import SFTTrainer

# ================================================================
# 2. GİRİŞ VE MODEL (STANDART)
# ================================================================
print("🛑 HUGGING FACE TOKEN GİRİNİZ (Write yetkili):")
login()

MODEL_ID = "google/medgemma-4b-it"

print(f"\n🧠 Model Yükleniyor: {MODEL_ID}")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

try:
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_ID,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True,
        token=True
    )
    tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, token=True)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"
    print("✅ Model Hazır.")
except Exception as e:
    print("\n❌ MODEL İNDİRİLEMEDİ! Lütfen Token'ın doğru olduğundan ve lisansı kabul ettiğinden emin ol.")
    raise e

model = prepare_model_for_kbit_training(model)

# ================================================================
# 3. VERİ İŞLEME (GARANTİLİ YÖNTEM)
# ================================================================
DOSYA_ADI = "/content/medsim_case_v2.1.json"

if not os.path.exists(DOSYA_ADI):
    print("⚠️ Dosya bulunamadı. Lütfen yükleyin:")
    from google.colab import files
    uploaded = files.upload()
    DOSYA_ADI = list(uploaded.keys())[0]

with open(DOSYA_ADI, 'r', encoding='utf-8') as f:
    raw_data = json.load(f)

# Lab verisi düzeltici
def format_laboratuvar(lab_data):
    if not lab_data or lab_data == "Normal": return "Özellik yok."
    if isinstance(lab_data, str): return lab_data
    lab_text = ""
    if isinstance(lab_data, dict):
        for k, v in lab_data.items():
            lab_text += f"\n- {k}: {v}"
    return lab_text

# Prompt Şablonu
medsim_template = """Aşağıda bir tıbbi simülasyon görevi tanımlanmıştır.

### Instruction:
Sen tıp eğitimi için 'Sanal Hasta Simülasyonu' oluşturan bir yapay zekasın.
Görevin, '{tani}' tanısı için **Simülasyon Senaryo Kartı** hazırlamaktır.

KURALLAR:
1. **HASTA ROLÜ:** Hastanın adı, yaşı ve doktora söyleyeceği ilk şikayet cümlesini "Halk Ağzı" ile yaz.
2. **GİZLİ TIBBİ VERİ:** Doktorun muayene ve tetkik butonlarına bastığında göreceği bulguları "Tıbbi Dil" ile eksiksiz yaz.
3. Çıktı JSON değil, aşağıdaki başlıklarla ayrılmış okunaklı bir rapor olmalı.

### Input:
Yeni vaka oluştur.

### Response:
{output}""" + tokenizer.eos_token

# Veriyi Metne Çevirme
processed_data = []
for vaka in raw_data:
    try:
        tani = vaka.get("gizli_tani", "Bilinmeyen")
        k = vaka.get("hasta_kimlik", {})
        a = vaka.get("anamnez", {})
        b = vaka.get("bulgular", {})
        lab = format_laboratuvar(b.get("laboratuvar", ""))

        output_text = f"""
=== KİMLİK ===
**Ad:** {k.get('ad_soyad','?')} ({k.get('yas','?')}, {k.get('cinsiyet','?')})
**Meslek:** {k.get('meslek','?')}

=== HASTA ROLÜ ===
"{k.get('sikayet','...')}"

=== TIBBİ VERİ ===
**Öykü:** {a.get('sikayet_detaylari','-')}
**Özgeçmiş:** {a.get('tibbi_ozgecmis','-')}
**İlaçlar:** {a.get('kullandigi_ilaclar','-')}
**Fizik Muayene:** {b.get('fizik_muayene','-')}
**Laboratuvar:** {lab}
**Görüntüleme:** {b.get('goruntuleme','-')}
"""
        full_text = medsim_template.format(tani=tani, output=output_text)
        processed_data.append({"text": full_text})
    except:
        continue

dataset = Dataset.from_list(processed_data)
print(f"✅ {len(dataset)} vaka eğitime hazır.")

# ================================================================
# 4. EĞİTİM (KARARLI AYARLAR)
# ================================================================
peft_config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"]
)

model = get_peft_model(model, peft_config)

class SmartBrakeCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs and 'loss' in logs:
            if logs['loss'] < 0.38:
                print(f"\n🛑 Loss {logs['loss']:.4f} oldu. Eğitim bitiyor.")
                control.should_training_stop = True

# --- KRİTİK KISIM: SFTConfig KULLANMIYORUZ, STANDART TRAININGARGS ---
# TRL 0.8.6 versiyonunda parametreleri buraya değil, aşağıya (SFTTrainer'a) yazarız.
training_args = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=60,
    num_train_epochs=2,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    optim="paged_adamw_8bit",
    output_dir="medsim_final_training",
    gradient_checkpointing=True,
    torch_compile=False # Hata önleyici
)

# SFTTrainer (Eski ama sağlam syntax)
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    dataset_text_field="text",  # TRL 0.8.6'da burası zorunludur
    max_seq_length=2048,        # TRL 0.8.6'da burası zorunludur
    packing=False,
    args=training_args,
    callbacks=[SmartBrakeCallback()]
)

print("🚀 EĞİTİM BAŞLIYOR... (Başarılı olacak)")
trainer.train()

# ================================================================
# 5. KAYDETME
# ================================================================
print("\n💾 Kaydediliyor...")
model.save_pretrained("MedSim_Final_Adapter")
print("✅ Adaptörler kaydedildi. İstersen bilgisayarına indirebilirsin.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.0/521.0 kB 26.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.57.3 requires huggingface-hub<1.0,>=0.34.0, but you have huggingface-hub 1.2.2 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 87.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.6/179.6 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 16.7 MB/s 


🧠 Model Yükleniyor: google/medgemma-4b-it


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/2.47k [00:00<?, ?B/s]


❌ MODEL İNDİRİLEMEDİ! Lütfen Token'ın doğru olduğundan ve lisansı kabul ettiğinden emin ol.


ValueError: The checkpoint you are trying to load has model type `gemma3` but Transformers does not recognize this architecture. This could be because of an issue with the checkpoint, or because your version of Transformers is out of date.

In [1]:
# ================================================================
# 1. NİHAİ KURULUM (BLEEDING EDGE - GELİŞTİRİCİ SÜRÜMÜ)
# ================================================================
# Standart sürümü siliyoruz ve GitHub'dan en son geliştirici sürümünü çekiyoruz.
# "gemma3" gibi yeni mimariler sadece burada var.
!pip uninstall -y transformers
!pip install git+https://github.com/huggingface/transformers
!pip install -q -U peft bitsandbytes trl accelerate huggingface_hub

from huggingface_hub import login
from google.colab import files
import torch
import json
import os
from datasets import Dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    TrainerCallback
)
from trl import SFTTrainer

# ================================================================
# 2. GİRİŞ VE MODEL YÜKLEME
# ================================================================
print("🛑 TOKEN GİRİŞİ (Write Yetkili):")
login()

MODEL_ID = "google/medgemma-4b-it" # Eğer link doğruysa bu çalışacak

print(f"\n🧠 Model Yükleniyor (Geliştirici Sürümüyle): {MODEL_ID}")

# 4-Bit Sıkıştırma
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

try:
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_ID,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True,
        token=True
    )
    tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, token=True)
    tokenizer.pad_token = tokenizer.eos_token
    print("✅ Model Başarıyla Yüklendi! (Sonunda...)")
except Exception as e:
    print(f"\n❌ KRİTİK HATA: Model yine yüklenemedi.")
    print(f"Hata Detayı: {e}")
    # Eğer bu da çalışmazsa bu model bozuktur veya public değildir.
    raise e

model = prepare_model_for_kbit_training(model)

# ================================================================
# 3. VERİ İŞLEME (KIRILMAZ YÖNTEM)
# ================================================================
DOSYA_ADI = "/content/medsim_case_v2.1.json"

if not os.path.exists(DOSYA_ADI):
    print("⚠️ Dosya bulunamadı. Lütfen yükleyin:")
    uploaded = files.upload()
    DOSYA_ADI = list(uploaded.keys())[0]

with open(DOSYA_ADI, 'r', encoding='utf-8') as f:
    raw_data = json.load(f)

# Lab verisi düzeltici
def format_laboratuvar(lab_data):
    if not lab_data or lab_data == "Normal": return "Özellik yok."
    if isinstance(lab_data, str): return lab_data
    lab_text = ""
    if isinstance(lab_data, dict):
        for k, v in lab_data.items():
            lab_text += f"\n- {k}: {v}"
    return lab_text

# Prompt Şablonu
medsim_template = """Aşağıda bir tıbbi simülasyon görevi tanımlanmıştır.

### Instruction:
Sen tıp eğitimi için 'Sanal Hasta Simülasyonu' oluşturan bir yapay zekasın.
Görevin, '{tani}' tanısı için **Simülasyon Senaryo Kartı** hazırlamaktır.

KURALLAR:
1. **HASTA ROLÜ:** Hastanın adı, yaşı ve doktora söyleyeceği ilk şikayet cümlesini "Halk Ağzı" ile yaz.
2. **GİZLİ TIBBİ VERİ:** Doktorun muayene ve tetkik butonlarına bastığında göreceği bulguları "Tıbbi Dil" ile eksiksiz yaz.
3. Çıktı JSON değil, aşağıdaki başlıklarla ayrılmış okunaklı bir rapor olmalı.

### Input:
Yeni vaka oluştur.

### Response:
{output}""" + tokenizer.eos_token

processed_data = []
for vaka in raw_data:
    try:
        tani = vaka.get("gizli_tani", "Bilinmeyen")
        k = vaka.get("hasta_kimlik", {})
        a = vaka.get("anamnez", {})
        b = vaka.get("bulgular", {})
        lab = format_laboratuvar(b.get("laboratuvar", ""))

        output_text = f"""
=== KİMLİK ===
**Ad:** {k.get('ad_soyad','?')} ({k.get('yas','?')}, {k.get('cinsiyet','?')})
**Meslek:** {k.get('meslek','?')}

=== HASTA ROLÜ ===
"{k.get('sikayet','...')}"

=== TIBBİ VERİ ===
**Öykü:** {a.get('sikayet_detaylari','-')}
**Özgeçmiş:** {a.get('tibbi_ozgecmis','-')}
**İlaçlar:** {a.get('kullandigi_ilaclar','-')}
**Fizik Muayene:** {b.get('fizik_muayene','-')}
**Laboratuvar:** {lab}
**Görüntüleme:** {b.get('goruntuleme','-')}
"""
        full_text = medsim_template.format(tani=tani, output=output_text)
        processed_data.append({"text": full_text})
    except:
        continue

dataset = Dataset.from_list(processed_data)
print(f"✅ {len(dataset)} vaka eğitime hazır.")

# ================================================================
# 4. EĞİTİM (STANDART AYARLAR)
# ================================================================
peft_config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"]
)

model = get_peft_model(model, peft_config)

class SmartBrakeCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs and 'loss' in logs:
            if logs['loss'] < 0.38:
                print(f"\n🛑 Loss {logs['loss']:.4f} oldu. Eğitim bitiyor.")
                control.should_training_stop = True

# Training Arguments
training_args = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=60,
    num_train_epochs=2,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    optim="paged_adamw_8bit",
    output_dir="medsim_final_training",
    gradient_checkpointing=True,
    torch_compile=False # Hata önleyici
)

# SFTTrainer
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=2048,
    packing=False,
    args=training_args,
    callbacks=[SmartBrakeCallback()]
)

print("🚀 EĞİTİM BAŞLIYOR...")
trainer.train()

# ================================================================
# 5. KAYDETME
# ================================================================
print("\n💾 Kaydediliyor...")
model.save_pretrained("MedSim_Final_Adapter")
print("✅ Tamamlandı.")

Found existing installation: transformers 4.57.3
Uninstalling transformers-4.57.3:
  Successfully uninstalled transformers-4.57.3
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-x1dcy260
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-x1dcy260
  Resolved https://github.com/huggingface/transformers to commit 64a7cc82a681803d6bdc9e96a1a212cebc5655d7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.0/521.0 kB 9.9 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-5.0.0.dev0-py3-none-any.whl size=10905000 sha256=cb4007429173ff5499abbf3aae4a3223db7b8a9014a5f0a4f872704bb9fc2aae
  Stored in directory: /tmp/pip-ephem-wheel-cache-w4wfhoh4/wheels/49/a7/50/c9fdabbf10e51bb1256adb0c1a587fedd7184f5bad28d47fe3
Successfully built transformers
  Attempt

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



🧠 Model Yükleniyor (Geliştirici Sürümüyle): google/medgemma-4b-it


config.json:   0%|          | 0.00/2.47k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/90.6k [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

✅ Model Başarıyla Yüklendi! (Sonunda...)


NotImplementedError: `get_input_embeddings` not auto‑handled for SiglipVisionTransformer; please override in the subclass.

In [1]:
# ================================================================
# 1. KURULUM (UNSLOTH İLE GARANTİ EĞİTİM)
# ================================================================
# Unsloth kütüphanesi Llama 3.1'i Colab T4'te eğitmenin TEK yoludur.
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers trl peft accelerate bitsandbytes

import torch
import json
import os
from unsloth import FastLanguageModel, is_bfloat16_supported
from datasets import Dataset
from trl import SFTTrainer
from transformers import TrainingArguments, TrainerCallback
from google.colab import files

# ================================================================
# 2. MODEL YÜKLEME (LLAMA 3.1 8B - TIP İÇİN EN İYİSİ)
# ================================================================
max_seq_length = 2048
dtype = None
load_in_4bit = True

# Unsloth'un optimize ettiği resmi Llama 3.1 modeli
MODEL_ID = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit"

print(f"🧠 Model Yükleniyor: {MODEL_ID} ...")

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = MODEL_ID,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

print("✅ Model Başarıyla Yüklendi! (Text-Based)")

# ================================================================
# 3. VERİ İŞLEME (JSON -> SENARYO METNİ)
# ================================================================
DOSYA_ADI = "/content/medsim_case_v2.1.json"

# Dosya yoksa yüklet
if not os.path.exists(DOSYA_ADI):
    print("⚠️ Dosya bulunamadı. Lütfen yükleyin:")
    uploaded = files.upload()
    DOSYA_ADI = list(uploaded.keys())[0]

try:
    with open(DOSYA_ADI, 'r', encoding='utf-8') as f:
        raw_data = json.load(f)
    print(f"✅ {len(raw_data)} vaka yüklendi. İşleniyor...")
except:
    raise ValueError("❌ JSON dosyası okunamadı!")

# Lab verisi düzeltici
def format_laboratuvar(lab_data):
    if not lab_data or lab_data == "Normal": return "Özellik yok."
    if isinstance(lab_data, str): return lab_data
    lab_text = ""
    if isinstance(lab_data, dict):
        for kategori, degerler in lab_data.items():
            lab_text += f"\n- [{kategori}]: "
            if isinstance(degerler, dict):
                items = [f"{k}: {v}" for k, v in degerler.items()]
                lab_text += ", ".join(items)
            else:
                lab_text += str(degerler)
    return lab_text

# Prompt Şablonu (Modelin rolünü belirliyoruz)
medsim_template = """Aşağıda bir tıbbi simülasyon görevi tanımlanmıştır.

### Instruction:
Sen tıp eğitimi için 'Sanal Hasta Simülasyonu' oluşturan bir yapay zekasın.
Görevin, '{tani}' tanısı için **Simülasyon Senaryo Kartı** hazırlamaktır.

KURALLAR:
1. **HASTA ROLÜ:** Hastanın adı, yaşı ve doktora söyleyeceği ilk şikayet cümlesini "Halk Ağzı" ile yaz.
2. **GİZLİ TIBBİ VERİ:** Doktorun muayene ve tetkik butonlarına bastığında göreceği bulguları "Tıbbi Dil" ile eksiksiz yaz.
3. Çıktı JSON değil, aşağıdaki başlıklarla ayrılmış okunaklı bir rapor olmalı.

### Input:
Yeni vaka oluştur.

### Response:
{output}""" + tokenizer.eos_token

# Veriyi Metne Çevirme Döngüsü
processed_data = []
for vaka in raw_data:
    try:
        tani = vaka.get("gizli_tani", "Bilinmeyen")
        k = vaka.get("hasta_kimlik", {})
        a = vaka.get("anamnez", {})
        b = vaka.get("bulgular", {})
        lab = format_laboratuvar(b.get("laboratuvar", ""))

        output_text = f"""
=== SİMÜLASYON KİMLİK KARTI ===
**Ad:** {k.get('ad_soyad','?')} ({k.get('yas','?')}, {k.get('cinsiyet','?')})
**Meslek:** {k.get('meslek','?')}

=== HASTA ROLÜ (CHAT BAŞLANGICI) ===
"{k.get('sikayet','...')}"

=== GİZLİ TIBBİ VERİTABANI ===
**[Öykü Detayı]:** {a.get('sikayet_detaylari','-')}
**[Özgeçmiş]:** {a.get('tibbi_ozgecmis','-')}
**[İlaçlar]:** {a.get('kullandigi_ilaclar','-')}

**[Fizik Muayene]:**
{b.get('fizik_muayene','-')}

**[Laboratuvar]:**{lab}

**[Görüntüleme]:**
{b.get('goruntuleme','-')}
"""
        full_text = medsim_template.format(tani=tani, output=output_text)
        processed_data.append({"text": full_text})
    except:
        continue

dataset = Dataset.from_list(processed_data)
print(f"✅ {len(dataset)} vaka eğitime hazır.")

# ================================================================
# 4. LoRA AYARLARI
# ================================================================
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth", # Llama 3 için bu ayar güvenlidir
    random_state = 3407,
)

# ================================================================
# 5. EĞİTİM (FREN SİSTEMLİ)
# ================================================================
class SmartBrakeCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs and 'loss' in logs:
            if logs['loss'] < 0.35: # Llama 3 çok hızlı öğrenir, 0.35 iyi bir frendir
                print(f"\n🛑 Loss {logs['loss']:.4f} oldu. Eğitim bitiyor.")
                control.should_training_stop = True

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 60,
        num_train_epochs = 2, # Tüm veriyi 2 tur dönsün
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "medsim_llama3_training",
    ),
    callbacks=[SmartBrakeCallback()]
)

print("🚀 EĞİTİM BAŞLIYOR... (Llama 3.1)")
trainer.train()

# ================================================================
# 6. KAYDETME
# ================================================================
print("\n💾 Model GGUF Formatında Kaydediliyor...")
model.save_pretrained_gguf("MedSim_Llama3_Final", tokenizer, quantization_method = "q4_k_m")
print("✅ TAMAMLANDI! Sol menüden 'MedSim_Llama3_Final-unsloth.Q4_K_M.gguf' dosyasını indir.")

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-a0t2bo4x/unsloth_50bc972e08184dcbbe1068fb987f1558
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-a0t2bo4x/unsloth_50bc972e08184dcbbe1068fb987f1558
  Resolved https://github.com/unslothai/unsloth.git to commit 8490f6efc407f409c42081988e93973df8e11f2d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.3/289.3 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.6/179.6 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 121.2 MB/s eta 0:00:00

model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

...

wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: buraktalha81 (buraktalha81-karadeniz-technical-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Detected [huggingface_hub.inference, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss


KeyboardInterrupt: 